# 🐩 Poodle Palette – Genotype → Phenotype Tutor
Welcome! In this notebook you’ll learn how two key genes (B and E) create the classic Poodle coat colours—and you’ll see the result for *any* genotype you enter.

In [1]:
# 📦 Setup
# If running locally, make sure pandas and matplotlib are installed.
# In Google Colab these come pre‑installed.
import pandas as pd
import matplotlib.pyplot as plt

# Display plots inline
%matplotlib inline


## 🎓 2. Genetic Background
Poodle colours hinge mainly on two loci:

| Locus | Dominant / recessive | What it controls |
|-------|----------------------|------------------|
| **B** | **B** = black pigment<br>**b** = brown pigment | `bb` turns all black areas brown (“liver”). |
| **E** | **E** = allows black/brown<br>**e** = blocks black → red/cream | `ee` dogs show only red/cream pigment. |

*We’ll ignore rarer loci (Dilute, Spotting) to keep the demo bite-sized.*

In [11]:
## 3. Imports + Helper Dictionaries


In [25]:
# 📦 Standard data-science imports
import pandas as pd
import matplotlib.pyplot as plt

# 📚 Teaching widgets (for the interactive allele picker we’ll add later)
from IPython.display import HTML
import ipywidgets as widgets

# 🖼️ Show charts inside the notebook
%matplotlib inline

# ────────────────────────────────────────────────────────────────
# Helper dictionaries for quick look-ups at each locus
# These will be used by predict_colour() in the next section.

# B-locus (black vs. brown pigment)
B_LOCUS = {
    "BB": "Black",
    "Bb": "Black",
    "bb": "Brown"
}

# E-locus (extension / red-cream override)
E_LOCUS = {
    "EE": "Allows black/brown",
    "Ee": "Allows black/brown",
    "ee": "Red/Cream (blocks black)"}

# Spotting / Parti – dominant solid S, recessive parti sp
S_LOCUS = {
    "SS": "solid",
    "Ssp": "solid (carrier)",
    "spsp": "parti"
}

# K-locus hierarchy  KB  >  Kbr  >  ky
K_HIERARCHY = ["KB", "Kbr", "ky"]    # first one present wins

# A-locus hierarchy  ay  >  at  >  aw  >  a
A_HIERARCHY = ["ay", "at", "aw", "a"]


In [14]:
## 4. Function `predict_colour()`

In [28]:
# ── K and A dominance tables ────────────────────────────────────────────
K_HIERARCHY = ["Kbr", "KB"]    # test for Kbr first, then KB; ky is the default
A_HIERARCHY = ["ay", "at", "aw", "a"]   # ay dominates at, etc.

def predict_colour(b_pair: str, e_pair: str,
                   k_pair: str, a_pair: str,
                   s_pair: str) -> str:
    """
    Translate five locus pairs into a coat-colour phrase.
    Inputs are *pairs*, e.g.  b_pair='Bb',  k_pair='Kbrky',  a_pair='atat'.
    """

    # 1️⃣  Base pigment from B & E
    if e_pair[0].islower() and e_pair[1].islower():          # ee → red/cream
        nose = "brown" if b_pair[0].islower() and b_pair[1].islower() else "black"
        colour = f"Red/Cream (nose: {nose})"
    else:                                                    # E_ → black / brown
        colour = "Brown" if b_pair[0].islower() and b_pair[1].islower() else "Black"

    # 2️⃣  K-locus winner
    if "Kbr" in k_pair:          # check Kbr first so it beats KB in 'Kbrky'
        k_dom = "Kbr"
    elif "KB" in k_pair:
        k_dom = "KB"
    else:
        k_dom = "ky"

    # 3️⃣  Pattern from K- & A-interaction
    pattern = ""
    if k_dom == "KB":            # dominant black masks A
        pattern = ""
    elif k_dom == "Kbr":         # brindle *unless* dog is recessive-black (a/a)
        dom_a = next(a for a in A_HIERARCHY if a in a_pair)
        pattern = "" if dom_a == "a" else "Brindle "
    else:                        # kyky → express true A pattern
        dom_a = next(a for a in A_HIERARCHY if a in a_pair)
        pattern = {
            "ay": "Sable ",
            "at": "Phantom ",
            "aw": "Agouti ",
            "a":  ""
        }[dom_a]

    # 4️⃣  Spotting / parti layer
    pattern_spot = "Parti " if "sp" in s_pair.lower() and "sp" in s_pair.lower()[-2:] else ""

    # 5️⃣  Assemble
    return f"{pattern_spot}{pattern}{colour}"


In [34]:
tests = [
    ("BB", "EE", "KBky", "ayat", "SS"),     # dominant black
    ("bb", "EE", "Kbrky", "ayat", "SS"),    # brindle sable (brown)
    ("BB", "EE", "kyky", "ataw", "spsp"),   # parti phantom
    ("bb", "EE", "kyky", "aya", "SS"),     #
    ("bb", "ee", "KBky", "ayay", "Ssp"),    # red/cream overrides
]
for t in tests:
    print(f"{t} → {predict_colour(*t)}")

('BB', 'EE', 'KBky', 'ayat', 'SS') → Black
('bb', 'EE', 'Kbrky', 'ayat', 'SS') → Brindle Brown
('BB', 'EE', 'kyky', 'ataw', 'spsp') → Parti Phantom Black
('bb', 'EE', 'kyky', 'aya', 'SS') → Sable Brown
('bb', 'ee', 'KBky', 'ayay', 'Ssp') → Parti Red/Cream (nose: brown)


In [8]:
## 5. Bulk Conversion of Your CSV

In [23]:
## 6. Interactive Playground

In [35]:
def show_interactive():
    # --- locus buttons ---
    b_btn = widgets.ToggleButtons(
        options=[("BB (black)", "BB"), ("Bb (black)", "Bb"), ("bb (brown)", "bb")],
        description="B locus:",
    )
    e_btn = widgets.ToggleButtons(
        options=[("EE (permits)", "EE"), ("Ee (permits)", "Ee"), ("ee (red)", "ee")],
        description="E locus:",
    )
    k_btn = widgets.ToggleButtons(
        options=[("KB", "KBKB"), ("Kbr", "Kbrky"), ("ky/ky", "kyky")],
        description="K locus:",
    )
    a_btn = widgets.ToggleButtons(
        options=[("ay/ay (sable)", "ayay"), ("at/at (phantom)", "atat"),
                 ("aw/aw (agouti)", "awaw"), ("a/a (rec.black)", "aa")],
        description="A locus:",
    )
    s_btn = widgets.ToggleButtons(
        options=[("SS (solid)", "SS"), ("Ssp (carrier)", "Ssp"),
                 ("sp/sp (parti)", "spsp")],
        description="S locus:",
    )

    out = widgets.Output()

    def update(_=None):
        result = predict_colour(
            b_btn.value, e_btn.value,
            k_btn.value, a_btn.value,
            s_btn.value
        )
        with out:
            out.clear_output()
            display(HTML(f"<h3>{result}</h3>"))

    # trigger on any change
    for w in (b_btn, e_btn, k_btn, a_btn, s_btn):
        w.observe(update, names="value")

    update()  # initial paint
    display(b_btn, e_btn, k_btn, a_btn, s_btn, out)

show_interactive()


ToggleButtons(description='B locus:', options=(('BB (black)', 'BB'), ('Bb (black)', 'Bb'), ('bb (brown)', 'bb'…

ToggleButtons(description='E locus:', options=(('EE (permits)', 'EE'), ('Ee (permits)', 'Ee'), ('ee (red)', 'e…

ToggleButtons(description='K locus:', options=(('KB', 'KBKB'), ('Kbr', 'Kbrky'), ('ky/ky', 'kyky')), value='KB…

ToggleButtons(description='A locus:', options=(('ay/ay (sable)', 'ayay'), ('at/at (phantom)', 'atat'), ('aw/aw…

ToggleButtons(description='S locus:', options=(('SS (solid)', 'SS'), ('Ssp (carrier)', 'Ssp'), ('sp/sp (parti)…

Output()

In [ ]:
## 7. Visual Summary

In [10]:
## 8. Further Reading